In [1]:
import numpy as np 
import pandas as pd 
 
import seaborn as sns 
import matplotlib.pyplot as plt

In [ ]:
file_path = ''
gps_gf = np.load(file_path)
print('gps data shape: ', gps_gf.shape)

gps_df = pd.DataFrame(gps_gf[1:,:], columns=gps_gf[0,:])
cols_to_convert = gps_df.columns[gps_df.columns != 'interval_id']
gps_df[cols_to_convert] = gps_df[cols_to_convert].apply(pd.to_numeric, errors='coerce') 

gps_df.shape

In [ ]:
def rotation_ned(lat, lon):
    R_ned = np.array([[-np.sin(lat)*np.cos(lon), -np.sin(lat)*np.sin(lon), np.cos(lat)],
                     [-np.sin(lon), np.cos(lon),0],
                     [-np.cos(lat)*np.cos(lon), -np.cos(lat)*np.sin(lon), -np.sin(lat)]])
    return R_ned 
def gps_pos_vel(gps_ned, dt): 

    vel_ned = gps_ned / dt.values[:, np.newaxis]
    acc_ned = np.diff(vel_ned, axis=0) / dt.values[1:, np.newaxis]  
    displacement = np.cumsum(gps_ned, axis=0)
    return vel_ned, acc_ned, displacement

def ecef_to_ned(gps_pos_ref):
    deltas = (gps_pos_ref[['Node[1].ECEF.X', 'Node[1].ECEF.Y', 'Node[1].ECEF.Z']].values /1000) - (gps_pos_ref[['Node[1].ECEF.X.ref', 'Node[1].ECEF.Y.ref', 'Node[1].ECEF.Z.ref']].values/1000)   
    
    #use reference lat and lon points 
    lons = gps_pos_ref['Node[1].LLH.Lon.ref'] * (np.pi/180) 
    lats = gps_pos_ref['Node[1].LLH.Lat.ref'] * (np.pi/180)

    ned_values = []
    mags = []
    timestamps = []
    for i, (lat, lon) in enumerate(zip(lats, lons)): 
        rotation_matrix = rotation_ned(lat,lon)
        norm = np.linalg.norm(deltas[i,:])
        ned_vec = rotation_matrix @ deltas[i,:] / norm
        ned_mag = np.linalg.norm(ned_vec)
        mags.append(ned_mag )
        ned_values.append(ned_vec * norm)
        timestamps.append(gps_pos_ref.iloc[i]['timestamp'])
    ned_values = np.array(ned_values)
    return ned_values, np.array(mags), np.array(timestamps)

new_gps = gps_df.copy()
new_gps[['Node[1].ECEF.X.ref','Node[1].ECEF.Y.ref', 'Node[1].ECEF.Z.ref',
         'Node[1].LLH.Lat.ref','Node[1].LLH.Lon.ref','timestamp_ref']] = new_gps[["Node[1].ECEF.X", "Node[1].ECEF.Y",
                                                                                  "Node[1].ECEF.Z",'Node[1].LLH.Lat',
                                                                                  'Node[1].LLH.Lon', 'timestamp']].shift(1)

gps_pos_ref = new_gps[["Node[1].ECEF.X", "Node[1].ECEF.Y",
                       "Node[1].ECEF.Z",'Node[1].ECEF.X.ref',
                       'Node[1].ECEF.Y.ref', 'Node[1].ECEF.Z.ref',
                      'Node[1].LLH.Lat.ref', 'Node[1].LLH.Lon.ref',
                      'timestamp','timestamp_ref','interval_id']].dropna()

gps_ned, ned_mags, timestamps = ecef_to_ned(gps_pos_ref) 
print(gps_ned.shape, timestamps.shape)

dt = gps_pos_ref['timestamp'] - gps_pos_ref['timestamp_ref']
vel_ned, acc_ned, displacement = gps_pos_vel(gps_ned, dt)
print(vel_ned.shape, acc_ned.shape, displacement.shape)
gps_vel_pos = np.column_stack([vel_ned, displacement, timestamps])

gps_vel_pos_df = pd.DataFrame(gps_vel_pos, columns=['vel.x', 'vel.y', 'vel.z', 'pos.x', 'pos.y', 'pos.z', 'timestamp'])

In [ ]:
xl_global = pd.read_csv(r"C:\Users\johncmarkowicz\BaseStation\AI\notebooks\ai_models\ai_models_data\node1_global_xl_combined.npy")
xl_global.head()

In [ ]:
new_ts_col = []
curr_ts = 0
dt = 1/1000
curr_int = 0
for _, row in xl_global.iterrows():
    if curr_ts == 0:
        curr_ts = float(int(row['timestamp']))
        curr_int = int(row['timestamp'])
        new_ts_col.append(curr_ts)

    else:
        if curr_int != int(row['timestamp']):
            if int(row['timestamp']) - curr_int > 1:
                new_ts = round(new_ts_col[-1] + 1 + dt, 3)
                new_ts_col.append(new_ts)
            else:    
                new_ts = round(new_ts_col[-1] + dt, 3)
                new_ts_col.append(new_ts)
            
            curr_int = int(row['timestamp'])
            continue

        new_ts = round(new_ts_col[-1] + dt, 3)
        new_ts_col.append(new_ts)

xl_global['timestamp_aligned'] = pd.Series(new_ts_col)
xl_global['timestamp_aligned'].head()

np.unique(np.diff(xl_global['timestamp'].astype(int)), return_counts=True) # when dt is 2, this indicates missing interval 

In [ ]:
def get_acc(row):
    acc = np.sqrt(row['x']**2 + row['y']**2)
    if row['x'] < 0:
        acc*=-1
    else:
        acc = np.abs(acc)
    return acc

def xl_state_estimate(x_prev, u, F, dt):
    x_k = F @ np.array(x_prev) + np.array([(dt**2)/2, dt]) * u 
                   
    return x_k 


def Kalman_acc_update(x_prev, P_prev, Q_xl, F, row, imu_dt):

    acc = get_acc(row)
    x_update = xl_state_estimate(x_prev, acc, F, imu_dt)
    P_update = F @ P_prev @ F.T + Q_xl

    return x_update, P_update


def Kalman_gps_udpate(gps_state, P_prev, x_prev, Q_gps):
    
    K = P_prev @ np.linalg.inv(P_prev + Q_gps)
    H = np.array([[1, 0], [0, 1]])  #
    # print("GPS STATE,XPREV",gps_state, x_prev)
    # print("GAIN ", K)
    # print("Kalman Resiudal",  K @ (gps_state - x_prev))
    # print("RESIDUAL: ", gps_state - x_prev)
    # print("GPS STATE", gps_state)
    x_update = x_prev + K @ (gps_state - x_prev)
    # P_update = P_prev - K @ Q_gps
    P_update = (np.eye(2) - K @ H) @ P_prev

    # print("GPS UPDATE", x_update)
    return x_update, P_update, K

def test_kalman(xl_global, gps_df, init_x, gps_dt):
    
    gps_idx = 0 
    x_history = [init_x]
    P_history = [Q_xl]
    gain_history = []

    F = np.array([[1, .001],[0, 1]]) 

    for i, row in xl_global.iterrows():

        if i == 0:
            imu_dt = .001
            gps_dt = .2

        else: 
            imu_dt = xl_global.iloc[i]['timestamp_aligned'] - xl_global.iloc[i-1]['timestamp_aligned']
            gps_dt = gps_df.iloc[gps_idx]['timestamp'] - gps_df.iloc[gps_idx-1]['timestamp']


        spectral_density_variance = 0.17527 # after 1 hour 
        velocity_random_walk_variance_xy =  0.0053 # after 1 hour 

        Q_xl = spectral_density_variance * np.array([
                [imu_dt**4 / 4, imu_dt**3 / 2],
                [imu_dt**3 / 2, imu_dt**2]
            ]) + velocity_random_walk_variance_xy * np.array([
                [0, 0],
                [0, 1]
            ])

        gps_vel_var = (0.125 * 5) * gps_dt
        gps_pos_var = (gps_vel_var * 5) * gps_dt**2

        R_gps = np.array([[gps_pos_var,0],[0, gps_vel_var]]) 

        xl_ts = row['timestamp_aligned']
        match = gps_df[gps_df['timestamp'] == xl_ts]
                    
        x_k, P_k = Kalman_acc_update(x_history[-1], P_history[-1], Q_xl, F, row, imu_dt)
        # print("X_K", x_k)

        if not match.empty and xl_ts != 46757790.0:
            gps_idx += 1
            vel_xyz = match[['vel.x', 'vel.y']]
            pos_xyz = match[['pos.x', 'pos.y']]
            gps_vel = np.linalg.norm(vel_xyz, axis=1)
            gps_pos = np.linalg.norm(pos_xyz, axis=1)
            gps_state = np.array([gps_pos[0], gps_vel[0]])
            # print("GPS STATE,XPREV",gps_state,  x_history[-1])

            x_update, P_update, gain = Kalman_gps_udpate(gps_state, P_k, x_k, R_gps)

            x_history.append(x_update)
            P_history.append(P_update)
            gain_history.append(gain)

        else:
            x_history.append(x_k)
            P_history.append(P_k)

    return np.array(x_history), np.array(P_history), gain_history


In [ ]:
inital_pos = np.linalg.norm([-20.153199, 36.532236]) # these come from getting the first xl and gps shared timestamp 
inital_vel = np.linalg.norm([-6.823665, 12.101643])
init_x = np.array([inital_pos, inital_vel])

x_history, P_history, gain_history = test_kalman(xl_global.iloc[:], gps_vel_pos_df.iloc[:], init_x, gps_dt=0.2)

In [ ]:
p_hist = np.array(P_history)
cov_00 = []
cov_01 = []

cov_11= []
cov_10= []

for p in p_hist:
    cov_00.append(p[0,0])
    cov_01.append(p[0,1])

    cov_11.append(p[1,1])
    cov_10.append(p[0,1])

plt.plot(cov_00)
plt.show()

plt.plot(cov_01)
plt.show()

plt.plot(cov_10)
plt.show()

plt.plot(cov_11)
plt.show()


plt.plot(x_history[:,0])
plt.title('Position')
plt.show()
plt.plot(x_history[:,1])
plt.title('Velocity')
plt.show()

gain_0 = []
gain_1 = []
for k in gain_history:
    gain_0.append(k[0,0])
    gain_1.append(k[1,1])

plt.plot(gain_0)
plt.title('Kalman Gain')
plt.show()

plt.plot(gain_1)
plt.title('Kalman Gain')

plt.show()